## Big Brother Is Watching You: A Closer look at Backdoor construction

include the library pycryptodome


In [8]:
from Crypto.Cipher import AES
from Crypto.Hash import SHAKE128

fix n value, State size, Number of Feistel branches

lines: INPUT, 1

In [9]:
n = 4 # Number of Feistel rounds
size = 128 # State size
branch = 2 # Number of Feistel branches
key=input("Enter key")
if key=="def":
    key=0x3cf28000471a74d22d81b6c98a3dbe33
else :
    key=hex(key)
backdoor=input("Enter backdoor")
if backdoor=="def":
    backdoor=0x1831bcaa5bee08e639864e4823183090
else :
    backdoor=hex(backdoor)
plaintext=(int(input("Enter plaintext"),base=16))

some essential functions

In [10]:
## convert to bytes 
def conv_bytes(x):
    return int.to_bytes(x, byteorder="little", signed=False, length=size//8)
## convert to int 
def conv_int(x):
    return int.from_bytes(x, byteorder="little", signed=False)

##fns
def pad_zero(z, l_nibble):
    return (hex(z)[2:]).zfill(l_nibble)

def gen_fe_round_key(k, n):
    K = []
    for i in range(0, n):
        K.append(conv_int(get_enc(Ky=key, msg=conv_bytes(i))))
    
    return K

def truncate(data):
    data_hex = pad_zero(data, l_nibble=size//4) 
    return int('0x'+data_hex[0:size//(branch*4)], 16), int('0x'+data_hex[size//(branch*4):], 16)

def get_hash(msg): 
    H = SHAKE128.new()
    H.update(data=conv_bytes(msg))
    return conv_int(H.read(size//8)) # Hash output is fixed at `size` bits

def get_enc(msg, Ky):
    E = AES.new(Ky, AES.MODE_ECB)
    return E.encrypt(msg)

In [11]:
plaintext = conv_bytes(plaintext) 
key = conv_bytes(key) 
backdoor = conv_bytes(backdoor) 


# Get Feistel round keys
F_round_keys = gen_fe_round_key(k=key, n=n)

# Get post-whitening keys
KL, KR = truncate(data=conv_int(key))
p0, p1 = truncate(data=conv_int(backdoor))

Hp = get_hash(msg=p0), get_hash(msg=p1)

C0, C1 = truncate(data=conv_int(plaintext))

# Feistel round operation
for i in range(n):
    j = i % 2

    alpha = truncate(conv_int(get_enc(Ky=conv_bytes(F_round_keys[i]), msg=conv_bytes(Hp[j]))))[0]
    
    beta = truncate(conv_int(get_enc(Ky=conv_bytes(F_round_keys[i]), msg=conv_bytes(get_hash(C0)))))[0]

    f = alpha ^ beta # Feistel round function

    C1 = f ^ C1 # Update right branch

    if i < n-1: # Do not swap branches in last round
        C0, C1 = C1, C0 # Swap branches
    
# Post-whiten
C0, C1 = C0 ^ KL, C1 ^ KR


# Display output
plaintext = pad_zero(conv_int(plaintext), l_nibble=size//4) # Hexadecimal string
key = pad_zero(conv_int(key), l_nibble=size//4) # Hexadecimal string
backdoor = pad_zero(conv_int(backdoor), l_nibble=size//4) # Hexadecimal string
ciphertext = pad_zero(C0, l_nibble=size//(branch*4)) + pad_zero(C1, l_nibble=size//(branch*4)) # Hexadecimal string

print ('Key:\t\t', key)
print ('Backdoor Entry:\t', backdoor)
print ('Plaintext:\t', plaintext)
print ('Ciphertext:\t', ciphertext)

if backdoor == plaintext:
    if n % 2 == 0: # Even round
        l, r = p1, p0
    else: # Odd round
        l, r = p0, p1
    
    K_recovered = pad_zero(z = C0 ^ l, l_nibble=size//(branch*4)) + pad_zero(z = C1 ^ r, l_nibble=size//(branch*4)) # Hexadecimal string
    
    if  K_recovered == key:
        print('\033[31mRecovered Key:\t', (K_recovered), end='\033[0m\n')
    else:
        assert False

Key:		 3cf28000471a74d22d81b6c98a3dbe33
Backdoor Entry:	 1831bcaa5bee08e639864e4823183090
Plaintext:	 9eac455e039a58928e163658e1493a20
Ciphertext:	 7a7700e9f5f4f974e9ba2834160cf284
